In [1]:

import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
import warnings
import seaborn as sns
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')


c:\Users\gaomi\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
data=pd.read_csv("individual_book_train\stock_0.csv")
data.head()

,time_id,seconds_in_bucket,bid_price1,ask_price1,bid_price2,ask_price2,bid_size1,ask_size1,bid_size2,ask_size2,stock_id
0,5,0,1.001422,1.002301,1.00137,1.002353,3,226,2,100,0
1,5,1,1.001422,1.002301,1.00137,1.002353,3,100,2,100,0
2,5,5,1.001422,1.002301,1.00137,1.002405,3,100,2,100,0
3,5,6,1.001422,1.002301,1.00137,1.002405,3,126,2,100,0
4,5,7,1.001422,1.002301,1.00137,1.002405,3,126,2,100,0


In [3]:
def log_return(list_stock_prices):
    return np.log(list_stock_prices).diff()
def realized_volatility(series):
    return np.sqrt(np.sum(series**2))

In [4]:
data['wap1'] =(data['bid_price1'] * data['ask_size1'] + data['ask_price1'] * data['bid_size1'])/(data['bid_size1'] + data['ask_size1'])
data['log_return1'] = data.groupby('time_id')['wap1'].transform(log_return)
data["BidAskSpread"] = data["ask_price1"] / data["bid_price1"] - 1

In [5]:
def flatten_name(prefix, src_names):
    ret = []
    for c in src_names:
        if c[0] in ['time_id', 'stock_id']:
            ret.append(c[0])
        else:
            ret.append('.'.join([prefix] + list(c)))
    return ret

In [6]:
data["time_bucket"]=data["seconds_in_bucket"].apply(lambda x:x//60)
data

,time_id,seconds_in_bucket,bid_price1,ask_price1,bid_price2,ask_price2,bid_size1,ask_size1,bid_size2,ask_size2,stock_id,wap1,log_return1,BidAskSpread,time_bucket
0,5,0,1.001422,1.002301,1.001370,1.002353,3,226,2,100,0,1.001434,NaN,0.000878,0
1,5,1,1.001422,1.002301,1.001370,1.002353,3,100,2,100,0,1.001448,0.000014,0.000878,0
2,5,5,1.001422,1.002301,1.001370,1.002405,3,100,2,100,0,1.001448,0.000000,0.000878,0
3,5,6,1.001422,1.002301,1.001370,1.002405,3,126,2,100,0,1.001443,-0.000005,0.000878,0
4,5,7,1.001422,1.002301,1.001370,1.002405,3,126,2,100,0,1.001443,0.000000,0.000878,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
917548,32767,568,0.998275,0.998754,0.997796,0.998946,90,90,48,28,0,0.998515,0.000434,0.000480,9
917549,32767,569,0.998275,0.998754,0.997892,0.998946,91,90,200,28,0,0.998516,0.000001,0.000480,9
917550,32767,571,0.998275,0.998754,0.997892,0.998946,91,90,100,28,0,0.998516,0.000000,0.000480,9
917551,32767,572,0.998275,0.998754,0.997892,0.998946,92,90,100,28,0,0.998517,0.000001,0.000480,9


In [7]:
data["num_order"]=data["bid_size1"] + data["ask_size1"] + data["bid_size2"] + data["ask_size2"]

In [8]:
add_dict={"wap1":[np.mean],"num_order":[np.mean],"BidAskSpread":[np.mean],"log_return1":[realized_volatility]}
feature=data.groupby(["time_id","time_bucket"]).agg(add_dict).reset_index()
feature.columns = flatten_name('book', feature.columns)
feature

,time_id,book.time_bucket.,book.wap1.mean,book.num_order.mean,book.BidAskSpread.mean,book.log_return1.realized_volatility
0,5,0,1.002064,274.160000,0.000873,0.001649
1,5,1,1.003939,317.461538,0.000874,0.001151
2,5,2,1.003939,360.350000,0.000754,0.001994
3,5,3,1.004152,400.142857,0.000936,0.001525
4,5,4,1.003840,329.217391,0.000977,0.001083
...,...,...,...,...,...,...
38291,32767,5,0.998525,672.227273,0.000556,0.000756
38292,32767,6,0.998100,582.130435,0.000357,0.000334
38293,32767,7,0.998260,398.333333,0.000482,0.000602
38294,32767,8,0.998650,257.954545,0.000362,0.000561


In [9]:
feature["target"]=feature.groupby(["time_id"])["book.log_return1.realized_volatility"].shift(-1)
feature

,time_id,book.time_bucket.,book.wap1.mean,book.num_order.mean,book.BidAskSpread.mean,book.log_return1.realized_volatility,target
0,5,0,1.002064,274.160000,0.000873,0.001649,0.001151
1,5,1,1.003939,317.461538,0.000874,0.001151,0.001994
2,5,2,1.003939,360.350000,0.000754,0.001994,0.001525
3,5,3,1.004152,400.142857,0.000936,0.001525,0.001083
4,5,4,1.003840,329.217391,0.000977,0.001083,0.001637
...,...,...,...,...,...,...,...
38291,32767,5,0.998525,672.227273,0.000556,0.000756,0.000334
38292,32767,6,0.998100,582.130435,0.000357,0.000334,0.000602
38293,32767,7,0.998260,398.333333,0.000482,0.000602,0.000561
38294,32767,8,0.998650,257.954545,0.000362,0.000561,0.000955


In [10]:
feature[feature["book.time_bucket."]==8].isna().sum()

time_id                                 0
book.time_bucket.                       0
book.wap1.mean                          0
book.num_order.mean                     0
book.BidAskSpread.mean                  0
book.log_return1.realized_volatility    0
target                                  2
dtype: int64

In [11]:
def smape(y_true, y_pred):
    """
    Calculate Symmetric Mean Absolute Percentage Error (sMAPE) between true and predicted values.

    Arguments:
    y_true -- Array-like, true values
    y_pred -- Array-like, predicted values

    Returns:
    sMAPE -- Symmetric Mean Absolute Percentage Error
    """
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 2.0
    diff = np.abs(y_true - y_pred) / denominator
    return 100 * diff

In [12]:
from tqdm import tqdm
non_tradable=[]
for i in tqdm(range(0,127)):
    try:
        dir_path=f"individual_book_train/stock_{i}.csv"
        data=pd.read_csv(dir_path)
    except:
        non_tradable.append(i)

tradable_id=[i for i in range(0,127) if i not in non_tradable]
tradable_id

  0%|          | 0/127 [00:00<?, ?it/s]

100%|██████████| 127/127 [01:25<00:00,  1.49it/s]


[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 46,
 47,
 48,
 50,
 51,
 52,
 53,
 55,
 56,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 66,
 67,
 68,
 69,
 70,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 118,
 119,
 120,
 122,
 123,
 124,
 125,
 126]

In [13]:
import pandas as pd
import statsmodels.api as sm
import numpy as np


from joblib import Parallel, delayed
from tqdm import tqdm


def process_stock_baseline(stock_id):
    data = pd.read_csv(f"individual_book_train/stock_{stock_id}.csv")
    data['wap1'] = (data['bid_price1'] * data['ask_size1'] + data['ask_price1'] * data['bid_size1']) / (data['bid_size1'] + data['ask_size1'])
    data['log_return1'] = data.groupby('time_id')['wap1'].transform(log_return)
    data["BidAskSpread"] = data["ask_price1"] / data["bid_price1"] - 1
    data["num_order"] = data["bid_size1"] + data["ask_size1"] + data["bid_size2"] + data["ask_size2"]
    data["time_bucket"] = data["seconds_in_bucket"].apply(lambda x: x // 60)
    add_dict = {"wap1": [np.mean], "num_order": [np.mean], "BidAskSpread": [np.mean], "log_return1": [realized_volatility]}
    feature = data.groupby(["time_id", "time_bucket"]).agg(add_dict).reset_index()
    feature.columns = flatten_name('book', feature.columns)
    feature["target"] = feature.groupby(["time_id"])["book.log_return1.realized_volatility"].shift(-1)

    temp_performance = []

    for time_id in feature.time_id.unique():
        train_data = feature[(feature["time_id"] == time_id) & (feature["book.time_bucket."] < 8)][["book.wap1.mean", "book.num_order.mean", "book.BidAskSpread.mean", "target"]]
        train_data.dropna(subset=["target"],inplace=True)
        X_train = train_data[["book.wap1.mean", "book.num_order.mean", "book.BidAskSpread.mean"]]
        
        
        weights = 0.8 ** (np.arange(len(X_train) - 1, -1, -1) / 2)
       
        y_train = train_data['target']
        
       
        model = sm.WLS(y_train, X_train, weights=weights).fit()
        
        
        val_data = feature[(feature["time_id"] == time_id) & (feature["book.time_bucket."] == 8)][["book.wap1.mean", "book.num_order.mean", "book.BidAskSpread.mean", "target"]]
        X_val = val_data[["book.wap1.mean", "book.num_order.mean", "book.BidAskSpread.mean"]]
        y_val = val_data['target']
        
        
        predictions = model.predict(X_val)
        
        res = smape(y_val, predictions)
        temp_performance.append({"stock_id": stock_id, 'time_id': time_id, 'SMAPE': res})

    return pd.DataFrame(temp_performance)


results = Parallel(n_jobs=-1)(delayed(process_stock_baseline)(stock_id) for stock_id in tqdm(tradable_id))

performance_table = pd.concat(results, ignore_index=True)
print(performance_table)





100%|██████████| 112/112 [01:58<00:00,  1.05s/it]


        stock_id  time_id                 SMAPE
0              0        5    [6.86640562644329]
1              0       11  [108.76556621327565]
2              0       16  [121.34181715269105]
3              0       31   [38.12446083534874]
4              0       62   [86.80717799484664]
...          ...      ...                   ...
428927       126    32751  [42.496078788966116]
428928       126    32753   [71.86475860810681]
428929       126    32758   [83.10736025373569]
428930       126    32763   [58.46901319583395]
428931       126    32767   [37.13236324015288]

[428932 rows x 3 columns]


In [14]:
performance_table

,stock_id,time_id,SMAPE
0,0,5,[6.86640562644329]
1,0,11,[108.76556621327565]
2,0,16,[121.34181715269105]
3,0,31,[38.12446083534874]
4,0,62,[86.80717799484664]
...,...,...,...
428927,126,32751,[42.496078788966116]
428928,126,32753,[71.86475860810681]
428929,126,32758,[83.10736025373569]
428930,126,32763,[58.46901319583395]


In [15]:
performance_table['SMAPE'] = performance_table['SMAPE'].apply(lambda x: x[0] if x else None)


In [16]:
performance_table.rename({"SMAPE":"LAB4A_SMAPE"},axis=1,inplace=True)
performance_table

,stock_id,time_id,LAB4A_SMAPE
0,0,5,6.866406
1,0,11,108.765566
2,0,16,121.341817
3,0,31,38.124461
4,0,62,86.807178
...,...,...,...
428927,126,32751,42.496079
428928,126,32753,71.864759
428929,126,32758,83.107360
428930,126,32763,58.469013


In [17]:
final_performance=pd.read_csv("performance_table.csv",index_col=0).merge(performance_table,on=["stock_id","time_id"],how='left')
final_performance

,stock_id,time_id,LR_SMAPE,RF_SMAPE,LGB_SMAPE,ARIMA_SMAPE,LAB4A_SMAPE_x,GARCH_SMAPE,ENSMBLE_SMAPE,LAB4A_SMAPE_y
0,0,5,12.899735,10.540688,11.514050,57.032624,6.866406,13.931351,18.818472,6.866406
1,0,11,33.654015,69.384723,49.614546,111.912892,108.765566,86.239375,46.871247,108.765566
2,0,16,99.144909,81.522299,84.873387,141.777733,121.341817,121.666066,92.702955,121.341817
3,0,31,29.962603,27.836634,11.254558,137.578063,38.124461,11.823359,32.096065,38.124461
4,0,62,56.485850,69.431754,73.779680,46.936511,86.807178,38.013200,63.835470,86.807178
...,...,...,...,...,...,...,...,...,...,...
428680,126,32751,13.276665,16.497378,31.984591,49.672478,42.496079,2.771261,23.876709,42.496079
428681,126,32753,83.111193,82.895903,93.236932,66.598897,71.864759,37.610376,82.599003,71.864759
428682,126,32758,1.666130,1.806531,46.089555,200.000000,83.107360,19.427492,7.609154,83.107360
428683,126,32763,58.089380,46.223397,51.152453,58.231517,58.469013,55.845837,52.920737,58.469013


In [18]:
final_performance.to_csv("performance_table.csv")

In [19]:
final_performance.mean()

stock_id            62.442409
time_id          16039.210567
LR_SMAPE            29.708473
RF_SMAPE            29.248490
LGB_SMAPE           29.288335
ARIMA_SMAPE         58.434181
LAB4A_SMAPE_x       36.976449
GARCH_SMAPE         31.572893
ENSMBLE_SMAPE       28.955184
LAB4A_SMAPE_y       36.976449
dtype: float64